In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

from utils.utils import *
from utils.constants import *

# Data

To make valid comparison across different methods, we split the original `stack_train` into new train and validation data sets.

In [3]:
# Import data


y_train = pd.read_csv(get_absolute_path('y_train.csv', 'data'))
y_test = pd.read_csv(get_absolute_path('y_test.csv', 'data'))


stack_train = pd.read_csv(get_absolute_path('stacked_X_tr.csv', 'data'))
stack_test  = pd.read_csv(get_absolute_path('stacked_X_te.csv', 'data'))



In [4]:
stack_train.columns

Index(['Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum)',
       'pH, water, unfiltered, field, standard units (Maximum)',
       'pH, water, unfiltered, field, standard units (Minimum)',
       'Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum)',
       'Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean)',
       'Dissolved oxygen, water, unfiltered, milligrams per liter (Maximum)',
       'Dissolved oxygen, water, unfiltered, milligrams per liter (Mean)',
       'Dissolved oxygen, water, unfiltered, milligrams per liter (Minimum)',
       'Temperature, water, degrees Celsius (Mean)',
       'Temperature, water, degrees Celsius (Minimum)',
       'Temperature, water, degrees Celsius (Maximum)', 'Date', 'Location_ID',
       'Year', 'Month', 'Week', 'Weekday'],
      dtype='object')

In [13]:
stack_train = stack_train.astype(column_data_extended_types)
stack_test = stack_test.astype(column_data_extended_types)

NameError: name 'column_data_extended_types' is not defined

In [6]:
stack_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15651 entries, 0 to 15650
Data columns (total 17 columns):
 #   Column                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                --------------  -----  
 0   Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum)  15651 non-null  float64
 1   pH, water, unfiltered, field, standard units (Maximum)                                                15651 non-null  float64
 2   pH, water, unfiltered, field, standard units (Minimum)                                                15651 non-null  float64
 3   Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum)  15651 non-null  float64
 4   Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean)   

In [7]:
stack_train.describe()

,"Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum)","pH, water, unfiltered, field, standard units (Maximum)","pH, water, unfiltered, field, standard units (Minimum)","Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum)","Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean)","Dissolved oxygen, water, unfiltered, milligrams per liter (Maximum)","Dissolved oxygen, water, unfiltered, milligrams per liter (Mean)","Dissolved oxygen, water, unfiltered, milligrams per liter (Minimum)","Temperature, water, degrees Celsius (Mean)","Temperature, water, degrees Celsius (Minimum)","Temperature, water, degrees Celsius (Maximum)"
count,15651.000000,15651.000000,15651.000000,15651.000000,15651.000000,15651.000000,15651.000000,15651.000000,15651.000000,15651.000000,15651.000000
mean,0.065207,0.889402,0.028976,0.043222,0.567845,0.858595,0.604889,0.578301,0.557072,0.536780,0.553200
std,0.161383,0.034803,0.120825,0.133254,0.120533,0.031062,0.146772,0.171843,0.204049,0.212293,0.190127
min,0.000526,0.576923,0.000334,0.000508,0.125000,0.719512,0.075758,0.031496,0.059375,0.025641,0.090116
25%,0.001891,0.871795,0.001631,0.001777,0.486842,0.841463,0.515152,0.480315,0.393750,0.365385,0.398256
50%,0.002632,0.897436,0.002220,0.002441,0.565789,0.853659,0.606061,0.590551,0.531250,0.509615,0.531977
75%,0.005458,0.910256,0.003536,0.004824,0.657895,0.878049,0.712121,0.708661,0.740625,0.724359,0.723837
max,1.000000,0.987179,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
stack_test['Year']

0        2018.0
1        2018.0
2        2018.0
3        2018.0
4        2018.0
          ...  
10429    2017.0
10430    2017.0
10431    2017.0
10432    2017.0
10433    2017.0
Name: Year, Length: 10434, dtype: object

# Feature Engineering

In [8]:
stack_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10434 entries, 0 to 10433
Data columns (total 17 columns):
 #   Column                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                --------------  -----  
 0   Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum)  10434 non-null  float64
 1   pH, water, unfiltered, field, standard units (Maximum)                                                10434 non-null  float64
 2   pH, water, unfiltered, field, standard units (Minimum)                                                10434 non-null  float64
 3   Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum)  10434 non-null  float64
 4   Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean)   

## NOTE: 

Pretend we have the best model already.

This is a TODO!

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Select numeric and categorical columns
numeric_columns = stack_train.select_dtypes(include=['float64']).columns
categorical_columns = [#'Date', 
                       'Location_ID',
                       'Year',
                       'Month',
                       'Week',
                       'Weekday']  # Add any categorical columns here

# Create preprocessing transformers
numeric_transformer = StandardScaler()  # You can use other scalers as well
categorical_transformer = OneHotEncoder(drop='first')  # Use one-hot encoding for categorical columns

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

# Fit the preprocessor on training data and transform both train and test data
X_train_preprocessed = preprocessor.fit_transform(stack_train)
X_test_preprocessed = preprocessor.transform(stack_test)

# Converting the preprocessed data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train_preprocessed, label=y_train)
dval = xgb.DMatrix(X_test_preprocessed, label=y_test)

ValueError: Found unknown categories ['2018.0', '2017.0'] in column 1 during transform

In [15]:
# Defining XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'max_depth': 7,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Training the XGBoost model
model_xgb = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'validation')], early_stopping_rounds=10)

# Making predictions
y_pred_xgb = model_xgb.predict(dval)

# Calculating RMSE
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print("XGBoost RMSE:", rmse_xgb)

[21:49:29] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[0]	validation-rmse:0.14855
[1]	validation-rmse:0.13389
[2]	validation-rmse:0.12068
[3]	validation-rmse:0.10880
[4]	validation-rmse:0.09815
[5]	validation-rmse:0.08857
[6]	validation-rmse:0.07997
[7]	validation-rmse:0.07222
[8]	validation-rmse:0.06529
[9]	validation-rmse:0.05908
[10]	validation-rmse:0.05351
[11]	validation-rmse:0.04849
[12]	validation-rmse:0.04402
[13]	validation-rmse:0.04004
[14]	validation-rmse:0.03647
[15]	validation-rmse:0.03330
[16]	validation-rmse:0.03047
[17]	validation-rmse:0.02797
[18]	validation-rmse:0.02573
[19]	validation-rmse:0.02378
[20]	validation-rmse:0.02206
[21]	validation-rmse:0.02057
[22]	validation-rmse:0.01927
[23]	validation-rmse:0.01814
[24]	validation-rmse:0.01716
[25]	validation-rmse:0.01629
[26]	validation-rmse:0.01554
[27]	validation-rmse:0.01491
[28]	validat